In [2]:
import jieba
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
filepath = "data/traindata.csv"
# 加载全部数据
alldata = pd.read_csv(filepath,header=None)
alldata.columns=["label","content"]
alldata.shape

(800000, 2)

In [7]:
# 随机抽取n条数据
data = alldata.sample(n=1000)
# data = alldata.iloc[:1000]

In [8]:
data["label"].value_counts()

0    902
1     98
Name: label, dtype: int64

In [9]:
def logtime(func):
    """
    函数目的：测量函数运行时间
    Parameter:
        func - 被测量的函数
    Return:
        wrapper - 被装饰之后的函数
    """
    def wrapper(*args,**kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        end = time.time()
        print("完成函数{name}, 运行时间 {totaltime:.3f}s".format(name=func.__name__,totaltime=end-start))
        start = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(start))
        end = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(end))
        print("开始时间 : %s \n结束时间 : %s "%(start,end))
        return result
    return wrapper

In [10]:
@logtime
def loadStopWords(filepath):
    """
    函数说明：加载获取停用词列表
    Parameter:
        filepath - 停用词文件的路径
    Return:
        stopwords - 以集合形式返回的停用词列表
    Modify:
        2017-12-02
    """
    stopwordfile = open(filepath,"r",encoding="utf-8").readlines()
    stopwords = {line.strip() for line in stopwordfile}
    return stopwords

In [664]:
stopwordsfile = "data/stopwords.txt"
stopwords = loadStopWords(stopwordsfile)

完成函数loadStopWords, 运行时间 0.001s
开始时间 : 2017-12-02 16:04:27 
结束时间 : 2017-12-02 16:04:27 


In [649]:
print(len(stopwords))

196


In [650]:
@logtime
def getVocalist(contents,stopwords):
    """
    函数说明：获取去除停用词后的词汇表，来减小向量维度的大小
    Parameter:
        data - 待处理的文本数据
        stopwords - 停用词列表
    Return:
        list(vocalist)- 文本词汇表
    Modify:
        2017-12-02
    """
    vocalist = set([])
    for i in range(len(data)):
        content = contents.iloc[i]
        segresult = set(jieba.cut(content))
        # 两个集合求并集
        vocalist = vocalist | segresult
    vocalist = vocalist - stopwords
    # print(vocalist)
    print("词汇表长度为：",len(vocalist))
    return list(vocalist)

In [651]:
contents = data["content"]
vocalist=getVocalist(contents,stopwords)   

词汇表长度为： 5564
完成函数getVocalist, 运行时间 0.795s
开始时间 : 2017-12-02 16:02:03 
结束时间 : 2017-12-02 16:02:03 


In [652]:
# 文本信息矢量化
def Vectorization(content,vocalist,stopwords):
    """
    函数说明：得到每个短信的0-1向量表示，如[0,0,0,1,0,1,0,0,0,0....]，向量长度与词汇表长度相同
    Paremeter:
        content -  每条短信的内容
        vocalist - 词汇表
        stopwords - 停用词列表
    Return:
        vector - 每条短信的向量表示
    Modify:
        2017-12-02
    """
    vector= [0]*len(vocalist)
    segresult = set(jieba.cut(content))-stopwords
    for word in segresult:
        vector[vocalist.index(word)] = 1
    return vector
@logtime
def vectall(data,vocalist,stopwords):
    data["vector"] = data.apply(lambda row:getVect(row["content"],vocalist,stopwords),axis=1)

In [654]:
# 得到每个文本的矢量化结果
vectall(data,vocalist,stopwords)

完成函数vectall, 运行时间 2.321s
开始时间 : 2017-12-02 16:02:32 
结束时间 : 2017-12-02 16:02:35 


C:\Developer\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [655]:
# 随机划分训练数据和测试数据 8:2
traindata,testdata = train_test_split(data,test_size=0.2)

In [656]:
print("训练数据的标签分布：")
print(traindata["label"].value_counts())
print("测试数据的标签分布：")
print(testdata["label"].value_counts())

训练数据的标签分布：
0    724
1     76
Name: label, dtype: int64
测试数据的标签分布：
0    172
1     28
Name: label, dtype: int64


In [657]:
# 构建分类器
@logtime
def trainclassifier(traindata):
    """
    函数说明：构建不同种类的分类器
    Parameter:
        traindata - 训练数据
    Return:
        classifier.fit(vector,label) - 训练好的分类器
    Modify:
        2017-12-02
    """
    classifier = svm.SVC(C=50,kernel='linear')
    # 需转换为list,不然报错
    vector = list(traindata["vector"])
    label = list(traindata["label"])
    return classifier.fit(vector,label)

In [658]:
classificer = trainclassifier(traindata)

完成函数trainclassifier, 运行时间 10.208s
开始时间 : 2017-12-02 16:03:45 
结束时间 : 2017-12-02 16:03:55 


In [659]:
# 应用分类器
@logtime
def applyClassificer(classificer,testdata):
    return classificer.predict(list(testdata["vector"]))
pred = applyClassificer(classificer,testdata)

完成函数applyClassificer, 运行时间 2.607s
开始时间 : 2017-12-02 16:03:58 
结束时间 : 2017-12-02 16:04:01 


In [660]:
def elevate_result(label,pred):
    """
    函数说明: 对分类器预测的结果进行评估，包括accurancy,precision,recall,F-score
    Parameter:
        label - 真实值
        pred - 预测值
    Return:
        None
    Modify:
        2017-12-02
    """
    con_mat = metrics.confusion_matrix(label,pred)
    TP = con_mat[1,1]
    TN = con_mat[0,0]
    FP = con_mat[0,1]
    FN = con_mat[1,0]
    
    accurancy = (TP+TN)/(TP+TN+FN+FP)
    precison = TP/(TP+FP)
    recall = TP/(TP+FN)
    beta = 1
    F_score = (1+pow(beta,2))*precison*recall/(pow(beta,2)*precison+recall)
    
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    print("accurancy: %s \nprecison: %s \nrecall: %s \nF-score: %s" % (accurancy,precison,recall,F_score))

In [661]:
label = testdata["label"]
elevate_result(label,pred)

TP: 27
TN: 172
FP: 0
FN: 1
accurancy: 0.995 
precison: 1.0 
recall: 0.964285714286 
F-score: 0.981818181818
